<a href="https://colab.research.google.com/github/pedrocast7/Speech_Recog_Models_Analysis/blob/main/Sci_Tools_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 - Installing Dependencies and Importing Used Libriries

In [1]:
!pip install transformers
!pip install wandb
!pip install plotly
!pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
## numerical and models processing
import torch
import numpy as np
import random

## for graphs and model tracking
import matplotlib.pyplot as plt
import plotly.express as px

## for dealing with data/files and instantiate models
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


# Importing NLTK for natural language processing
import nltk

from nltk.corpus import stopwords    # For stopwords

# Downloading NLTK data
nltk.download('stopwords')   # Downloading stopwords data
nltk.download('punkt')       # Downloading tokenizer data


import utils

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1 - Configurations for Reproducibility and Setting Device for GPU Acceleration

In [2]:
# Setting the random seed for reproducibility
seed = 77
np.random.seed(seed) ## for numpy
torch.manual_seed(seed) ## for PyTorch
torch.cuda.manual_seed(seed)
random.seed(77) ## for random module on python


# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("GPU is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## use gpu if its available

GPU is available.
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 4050 Laptop GPU


### 1.2 - Downloading Dataset from Kaggle

In [3]:
import kagglehub

# Download latest version
path_to_dataset = kagglehub.dataset_download("team-ai/spam-text-message-classification")

print(f'Path to dataset files: {path_to_dataset}')

Path to dataset files: C:\Users\pedro\.cache\kagglehub\datasets\team-ai\spam-text-message-classification\versions\1


## 2 - Data Preprocessing

In [4]:
df = pd.read_csv( path_to_dataset + '/SPAM text message 20170820 - Data.csv')



pd.set_option('display.max_colwidth', 800) ## see more text from message


#rename the columns name
df.rename(columns = {'Message': 'text'}, inplace = True)


#remove Duplicate
df = df.drop_duplicates(keep = 'first')

#encoding the categorical labels to 0-1 (binary classification)
encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['Category'])

df['num_characters'] = df['text'].apply(len)
df['num_words'] = df['text'].apply(lambda x: len(nltk.word_tokenize(x)))
df['num_sentence'] = df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))


df.head(10)

,Category,text,target,num_characters,num_words,num_sentence
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0,111,24,2
1,ham,Ok lar... Joking wif u oni...,0,29,8,2
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1,155,37,2
3,ham,U dun say so early hor... U c already then say...,0,49,13,1
4,ham,"Nah I don't think he goes to usf, he lives around here though",0,61,15,1
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",1,147,39,4
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,0,77,18,2
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,0,160,31,2
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.,1,157,32,5
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,1,154,31,3


In [6]:
#testing

utils.transform_text(df["text"][0])

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [ ]:
#preprocessing data using created function transform_text (remove punctuation, special characters, stopwords and steammer data, tokenize)
df['transformed_text'] = df['text'].apply(utils.transform_text)

In [8]:
df.head(10)

,Category,text,target,num_characters,num_words,num_sentence,transformed_text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0,111,24,2,go jurong point crazi avail bugi n great world la e buffet cine got amor wat
1,ham,Ok lar... Joking wif u oni...,0,29,8,2,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1,155,37,2,free entri 2 wkli comp win fa cup final tkt 21st may text fa 87121 receiv entri question std txt rate c appli 08452810075over18
3,ham,U dun say so early hor... U c already then say...,0,49,13,1,u dun say earli hor u c alreadi say
4,ham,"Nah I don't think he goes to usf, he lives around here though",0,61,15,1,nah think goe usf live around though
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",1,147,39,4,freemsg hey darl 3 week word back like fun still tb ok xxx std chg send rcv
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.,0,77,18,2,even brother like speak treat like aid patent
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,0,160,31,2,per request mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun
8,spam,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.,1,157,32,5,winner valu network custom select receivea prize reward claim call claim code kl341 valid 12 hour
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030,1,154,31,3,mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030


In [ ]:
#using vectorization for the processed data

from sklearn.feature_extraction.text import TfidfVectorizer
tfid = TfidfVectorizer(max_features = 3000)


X = tfid.fit_transform(df['transformed_text']).toarray()
y = df['target'].values